<a href="https://colab.research.google.com/github/chaitanya201003/AIML_LAB-IIIT-H-/blob/main/AIML_III_Module_01_Lab_02_Machine_Learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


In [ ]:
# Different validation set sizes
validation_sizes = [60 / 100, 40 / 100, 20 / 100, 10 / 100]

for val_size in validation_sizes:
    traindata, trainlabel, valdata, vallabel = split(
        alltraindata, alltrainlabel, val_size)
    valpred = NN(traindata, trainlabel, valdata)
    valAccuracy = Accuracy(vallabel, valpred)
    print(f"Validation accuracy with {val_size*100}% validation set size:", valAccuracy*100, "%")


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
answer:
The accuracy of the validation set in a machine learning model can be affected by changing the size of the validation set.

Increasing the Percentage of the Validation Set
When you increase the percentage of the validation set, the following effects might be observed:

Decreased Training Set Size: Since a larger portion of the data is reserved for validation, the training set becomes smaller. With less data to train on, the model may not learn as effectively, potentially leading to a decrease in model performance.

More Reliable Validation Accuracy: A larger validation set provides a more reliable estimate of the model's performance because the estimate is based on a larger and more diverse subset of data. This reduces the variance in the validation accuracy and gives a more stable measure of how the model is likely to perform on unseen data.

Potential Overfitting: If the training set becomes too small, the model might overfit the limited training data, which means it will perform well on the training set but poorly on the validation set and unseen test data.

Decreasing the Percentage of the Validation Set
When you decrease the percentage of the validation set, the following effects might be observed:

Increased Training Set Size: With more data available for training, the model has more information to learn from, which can improve the training process and potentially enhance the model's performance.

Less Reliable Validation Accuracy: A smaller validation set means the estimate of the model's performance is based on a smaller subset of data. This can lead to higher variance in the validation accuracy, making it a less reliable measure of the model's true performance on unseen data.

Reduced Overfitting Risk: With a larger training set, the model is less likely to overfit since it has more examples to generalize from, leading to potentially better performance on the validation and test sets.


2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
answer:
The size of the training and validation sets significantly influences the model's performance and the reliability of the validation set as a predictor for test set accuracy. A larger training set generally enhances the model's ability to learn and generalize, thereby improving performance on unseen test data. Conversely, a smaller training set can lead to underfitting and poorer test accuracy. Meanwhile, a larger validation set offers a more stable and reliable estimate of the model's performance, reducing variance in accuracy measurements, but at the cost of reducing the training set size.

Balancing the sizes of the training and validation sets is crucial. While a common split like 80-20 is often effective, the ideal split may vary depending on the dataset. Techniques like cross-validation can optimize the use of available data, ensuring both effective model training and reliable performance estimation. Experimenting with different splits helps in identifying the best balance for achieving optimal test set accuracy and robust validation performance.

3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
answer:
A good percentage to reserve for the validation set typically ranges between 20% and 30% of the total dataset. This range strikes a balance between having enough data to train the model effectively and ensuring a sufficiently large validation set to provide a reliable estimate of the model's performance.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Implementing 1-nearest neighbor classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train)
y_pred_1nn = knn1.predict(X_test)
accuracy_1nn = accuracy_score(y_test, y_pred_1nn)
print("Accuracy of 1-nearest neighbor classifier:", accuracy_1nn)

# Implementing 3-nearest neighbor classifier
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train, y_train)
y_pred_3nn = knn3.predict(X_test)
accuracy_3nn = accuracy_score(y_test, y_pred_3nn)
print("Accuracy of 3-nearest neighbor classifier:", accuracy_3nn)


Accuracy of 1-nearest neighbor classifier: 1.0
Accuracy of 3-nearest neighbor classifier: 1.0


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
answer:
Averaging validation accuracy across multiple splits provides more consistent and reliable results by reducing variance inherent in any single split, ensuring a robust estimate of the model's performance. This approach mitigates the impact of random variability, offers a better representation of the model's generalization ability across diverse data subsets, reduces sensitivity to specific data distributions, and enhances confidence in the reported accuracy. Overall, it's a preferred method for evaluating model performance as it provides a more stable and trustworthy assessment of the model's effectiveness.

2. Does it give more accurate estimate of test accuracy?
answer:
While averaging validation accuracy across multiple splits improves the reliability and stability of the estimated model performance, it doesn't directly provide a more accurate estimate of test accuracy. Instead, it offers a better approximation of how well the model is likely to generalize to unseen data based on the validation set.

The test accuracy, however, is ultimately determined by evaluating the model on a completely independent dataset that it has not seen during training or validation. Averaging validation accuracy across multiple splits helps in selecting a model with better generalization performance, which in turn may lead to a more accurate test accuracy estimate. However, it's important to note that validation accuracy and test accuracy are related but distinct measures. The true accuracy on unseen data can only be assessed through evaluation on the test set.

3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
answer:
While averaging validation accuracy across multiple splits improves the reliability and stability of the estimated model performance, it doesn't directly provide a more accurate estimate of test accuracy. Instead, it offers a better approximation of how well the model is likely to generalize to unseen data based on the validation set.

The test accuracy, however, is ultimately determined by evaluating the model on a completely independent dataset that it has not seen during training or validation. Averaging validation accuracy across multiple splits helps in selecting a model with better generalization performance, which in turn may lead to a more accurate test accuracy estimate. However, it's important to note that validation accuracy and test accuracy are related but distinct measures. The true accuracy on unseen data can only be assessed through evaluation on the test set.

4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
answer:
while increasing the number of iterations can help partially alleviate the challenges posed by small training or validation datasets, it may not fully compensate for the limitations in data size and diversity. It's still crucial to ensure that the datasets used for training, validation, and testing are sufficiently large and representative to enable the model to learn effectively and generalize well to unseen data.


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
iris = load_iris()
alldata = iris.data
alllabel = iris.target

# Define a function to compute accuracy
def compute_accuracy(alldata, alllabel, splitpercent, iterations, n_neighbors):
    accuracies = []
    for _ in range(iterations):
        # Split the data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(alldata, alllabel, test_size=1 - splitpercent)

        # Fit the KNN classifier
        knn = KNeighborsClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)

        # Predict labels on validation set
        y_pred = knn.predict(X_val)

        # Calculate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        accuracies.append(accuracy)

    # Return the average accuracy
    return np.mean(accuracies)

# Parameters
iterations = 10  # Number of iterations for each split
n_neighbors = 3  # Number of neighbors for KNN classifier

# List of split sizes to evaluate
split_sizes = [0.5, 0.6, 0.7, 0.8]

# List to store average accuracies for 3-nearest neighbor classifier
avg_accuracies_3nn = []

# List to store average accuracies for 1-nearest neighbor classifier
avg_accuracies_1nn = []

for split_size in split_sizes:
    # Compute average accuracy for 3-nearest neighbor classifier
    avg_accuracy_3nn = compute_accuracy(alldata, alllabel, split_size, iterations, n_neighbors)
    avg_accuracies_3nn.append(avg_accuracy_3nn)

    # Compute average accuracy for 1-nearest neighbor classifier
    avg_accuracy_1nn = compute_accuracy(alldata, alllabel, split_size, iterations, 1)
    avg_accuracies_1nn.append(avg_accuracy_1nn)

# Print the results
for i, split_size in enumerate(split_sizes):
    print(f"Split size: {split_size * 100}%")
    print(f"Average accuracy (3-nearest neighbor classifier): {avg_accuracies_3nn[i] * 100:.2f}%")
    print(f"Average accuracy (1-nearest neighbor classifier): {avg_accuracies_1nn[i] * 100:.2f}%")
    print()


Split size: 50.0%
Average accuracy (3-nearest neighbor classifier): 95.33%
Average accuracy (1-nearest neighbor classifier): 95.20%

Split size: 60.0%
Average accuracy (3-nearest neighbor classifier): 96.83%
Average accuracy (1-nearest neighbor classifier): 95.33%

Split size: 70.0%
Average accuracy (3-nearest neighbor classifier): 95.87%
Average accuracy (1-nearest neighbor classifier): 95.43%

Split size: 80.0%
Average accuracy (3-nearest neighbor classifier): 95.67%
Average accuracy (1-nearest neighbor classifier): 96.00%

